<a href="https://colab.research.google.com/github/jwalx/Generative-AI/blob/main/Generative_AI_LLM_practice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
!pip install torch
!pip install transformers
!pip install datasets

ERROR: Invalid requirement: '!pip'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.8 MB/s eta 0:00:00


# Loading the datasets, LLM, tokenizer and configurator

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c07c4cf4362c223c/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Printing a couple of dialogues with their summaries

In [4]:
example_indices  =[40,200]

dash_line = "-".join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print('INPUT DAILOGUE:')
  print(dataset["test"][index]['dialogue'])
  print(dash_line)
  print('BASELINE HUMAN SUMMARY:')
  print(dataset["test"][index]["summary"])
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DAILOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exam

# load the model `FLAN-T5`, creating an instance of the AutoModelSeq2SeqLM class with .from_pretrained() method.

In [5]:
model_name = "google/flan-t5-base"

model =AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

Test the tokenizer encoding and decoding

In [9]:
sentence = "what time is it,Tom?"

sentence_encoded = tokenizer(sentence,return_tensors = "pt")

sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens = True
)

print("Encoded Sentence:")
print(sentence_encoded["input_ids"][0])
print("Decoded SentenceL")
print(sentence_decoded)

Encoded Sentence:
tensor([ 125,   97,   19,   34,    6, 3696,   51,   58,    1])
Decoded SentenceL
what time is it,Tom?


# Summarize dialogue without an instruction prompt

In [11]:
for i, index in enumerate(example_indices):
  dialogue = dataset["test"][index]["dialogue"]
  summary = dataset["test"][index]["summary"]

  # adding prompt instead of dialogue
  inputs = tokenizer(dialogue , return_tensors = "pt")
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_token = True
  )

  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print(f"Input Prompt:\n{dialogue}")
  print(dash_line)
  print(f"Baseline human Summary:\n{summary}")
  print(dash_line)
  print(f"Model Generation - without Prompt Engineering:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation - without Prompt Engineering:
<pad> Person1: It's ten to nine.</s>

----------------------

# Summarize Dialogue with instruction Prompt

In [12]:
for i, index in enumerate(example_indices):
  dialogue = dataset["test"][index]["dialogue"]
  summary = dataset["test"][index]["summary"]

  prompt=f"""
Summarize the following conversation.

{dialogue}

Summary:
   """

  # adding prompt instead of dialogue
  inputs = tokenizer(prompt , return_tensors = "pt")
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_token = True
  )

  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print(f"Input Prompt:\n{dialogue}")
  print(dash_line)
  print(f"Baseline human Summary:\n{summary}")
  print(dash_line)
  print(f"Model Generation - without Prompt Engineering:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline human Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation - without Prompt Engineering:
<pad> The train is about to leave.</s>

--------------------

# One shot Inference



Let's build a function that takes a list of example_indices_full,generates a prompt with full examples, then at the end appends the prompt which you want to model to complete




In [16]:
def make_prompt(example_indices_full,example_index_to_summarize):
    prompt=""
    for index in example_indices_full:
      dialogue = dataset["test"][index]["dialogue"]
      summary = dataset["test"][index]["summary"]

      prompt +=f"""
Dialogue:

{dialogue}

What was going on :
{summary}

"""

    dialogue = dataset["test"][index]["dialogue"]

    prompt+=f"""

Dialogue:
{dialogue}

What was going on?

"""
    return prompt


In [17]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?




In [19]:
summary = dataset["test"][example_index_to_summarize]["summary"]

inputs = tokenizer(one_shot_prompt,return_tensors = "pt")
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f"Baseline human summary:\n{summary}\n")
print(dash_line)
print(f"model generation - one shot:\n{output}")


---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
model generation - one shot:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


# Few shot Inference

In [ ]:
example_indices_full = [40,80,120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)

print(few_shot_prompt)

In [ ]:
summary = dataset["test"][example_index_to_summarize]["summary"]

inputs = tokenizer(few_shot_prompt,return_tensors = "pt")
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0]
    skip_special_tokens =True
)

print(dash_line)
print(f"Baseline human summary:\n{summary}\n")
print(dash_line)
print(f"Model Generation - Few shot:\n{output}\n")